In [ ]:
import pandas as pd
import random
import torch

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
user_id = 1006
df = pd.read_csv("/content/drive/My Drive/BEAT_PD/Data/CIS-PD/training_data/HAR/merged/1006.csv",index_col=0)
df.head()

In [ ]:
#@title select one measurement
measurement_list = list(df['measurement_id'].unique())
measurement = measurement_list[21:]
measurement
# measurement = '0dfd3d96-7b99-4522-9bf5-c39046fcf096'


In [ ]:
start_point=0
window_size=250 #50Hz = 1s for SW, 100Hz = 1s for SP sensor

ts = df[df['measurement_id']==measurement]
    # start_point = random.randint(0,len(ts)-window_size) #return a random no between (0,len(ts)-window_size)
end_point = (start_point+window_size)-1
# print(end_point)
ts_slice = ts.iloc[start_point:end_point]  #creates a dataframe with data between start_point and end_point (i.e. creates df with 400 records)
ts_slice

#Function to get windowed data from Dataframe

In [ ]:

def select_df(df,measurement_id,start_point,window_size,target1,target2,target3):
    ts = df[df['measurement_id']==measurement_id]
    # start_point = random.randint(0,len(ts)-window_size) #return a random no between (0,len(ts)-window_size)
    end_point = start_point+window_size-1
    ts_slice = ts.iloc[start_point:end_point]  #creates a dataframe with data between start_point and end_point (i.e. creates df with 400 records)
    # target='on_off'
    value1 = ts[target1].unique()[0]
    value2 = ts[target2].unique()[0]
    value3 = ts[target3].unique()[0]
    picture_id = "{}to{}".format(start_point,end_point)
    return ts_slice, value1,value2,value3,picture_id

In [ ]:
# start_point=0
# window_size=400 #50Hz = 1s for SW, 100Hz = 1s for SP sensor

# ts_selected,val1,val2,val3,pid = select_df(df,measurement,start_point,window_size,'on_off','dyskinesia','tremor')
# print(val1,val2,val3,pid)


1.0 nan 1.0 0to249


#Plot selected windowed data

In [ ]:
col_names = ['X','Y','Z']
ts_p = ts_selected[col_names]
ts_p

Plot Time Series

In [ ]:
import matplotlib.pyplot as plt

axis = ts_p.plot(subplots=True,legend=False, figsize=(10,5))
for ax in axis:
  
  ax.tick_params(axis='x', colors='white')
  ax.tick_params(axis='y', colors='white')
  #ax.yticks(color='w')
  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)
  ax.spines['bottom'].set_visible(False)
  ax.spines['left'].set_visible(False)
  #ax.set_yticks([])
  #ax.set_xticks([])



#Function to save plots as img files

In [ ]:
import matplotlib.pyplot as plt

def plot_ts(save_path,measurement_id,target1,value1,target2,value2,target3,value3,picture_id):
  axis = ts_p.plot(subplots=True,legend=False,figsize=(20,5))
  for ax in axis:
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
  path = '{}/{}_{}_{}_{}_{}_{}_{}_{}.png'.format(save_path,measurement_id,target1,value1,target2,value2,target3,value3,picture_id)
  plt.savefig(path)
  print(path)


# Combining all and running whole for users

In [ ]:

from IPython.display import clear_output

target1 = 'on_off' 
target2 = 'dyskinesia'
target3 = 'tremor'
col_names = ['X','Y','Z']
#save_path = "/content/drive/My Drive/BEAT_PD/Data/CIS-PD/training_data/HAR/TS_Plots/1006"
save_path = "/content/drive/My Drive/BEAT_PD/Data/CIS-PD/training_data/HAR/TS_Plots/1006"
subject_id =1006
window_size=400


measurement_list = list(df['measurement_id'].unique())  # list of all the measurements
measurement_list = measurement_list[21:]
for i in range(len(measurement_list)):
  torch.cuda.empty_cache()
  measurement_id = measurement_list[i]
  ts = df[df['measurement_id']==measurement_id]
  # print(measurement_id)

  for start_point in range(0,len(ts.measurement_id),200):     # each loop will generate a picture and save it in save_path
    
    ts_slice, value1, value2, value3,picture_id = select_df(df,measurement_id,start_point,window_size,target1,target2,target3)
    ts_p = ts_slice[col_names]
    print(picture_id)
    clear_output()
    plot_ts(save_path,measurement_id,target1,value1,target2,value2,target3,value3,picture_id)

## Combining All functions in one cell for All users

In [ ]:
from IPython.display import clear_output
import glob
from os import listdir
from os.path import isfile, join
import pandas as pd
import random
import torch
import os
import tqdm
import gc
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pathlib
import time
plt.ioff()
import sys
clear = lambda: os.system('cls')
fd = os.open(os.devnull, os.O_RDWR)
# NB: even if stdin is closed, fd >= 0
os.dup2(fd, 1)
os.dup2(fd, 2)
if fd > 2:
  os.close(fd)

#Process all users in a loop - checks for processed files if it crashes
savedir = "/content/drive/My Drive/BEAT_PD/Data/CIS-PD/training_data/HAR/merged/"
files = [f for f in listdir(savedir) if isfile(join(savedir, f))]
files_user = [f for f in listdir(savedir) if isfile(join(savedir, f))]
#Output path
plot_path = "/content/drive/My Drive/BEAT_PD/Data/CIS-PD/training_data/HAR/TS_Plots/"

start_point=0
window_size=400 #50Hz = 1s for SW, 100Hz = 1s for SP sensor
overlap = 0.5 # 50%
overlap_win = int((window_size - start_point) * overlap)
target1 = 'on_off' 
target2 = 'dyskinesia'
target3 = 'tremor'
col_names = ['X','Y','Z']

def select_df(df,measurement_id,start_point,window_size,target1,target2,target3):
    ts = df[df['measurement_id']==measurement_id]
    # start_point = random.randint(0,len(ts)-window_size) #return a random no between (0,len(ts)-window_size)
    end_point = start_point+window_size-1
    ts_slice = ts.iloc[start_point:end_point]  #creates a dataframe with data between start_point and end_point (i.e. creates df with 400 records)
    # target='on_off'
    value1 = ts[target1].unique()[0]
    value2 = ts[target2].unique()[0]
    value3 = ts[target3].unique()[0]
    picture_id = "{}to{}".format(start_point,end_point)
    return ts_slice, value1,value2,value3,picture_id

def gen_plot_id(start_point,window_size):
    end_point = start_point+window_size-1
    picture_id = "{}to{}".format(start_point,end_point)
    return picture_id

def gen_labels(ts, save_path,measurement_id,target1,target2,target3, overlap_win):
  all_plot_names = []
  value1 = ts[target1].unique()[0]
  value2 = ts[target2].unique()[0]
  value3 = ts[target3].unique()[0]
  for start_point in range(0,len(ts.measurement_id),overlap_win):
    picture_id = gen_plot_id(start_point,window_size)
    path = '{}{}_{}_{}_{}_{}_{}_{}_{}.png'.format(save_path,measurement_id,target1,value1,target2,value2,target3,value3,picture_id)
    all_plot_names.append(path)
  return all_plot_names

def plot_ts(ts, save_path,measurement_id,target1,value1,target2,value2,target3,value3,picture_id):
  axis = ts.plot(subplots=True,legend=False,figsize=(20,5))
  for ax in axis:
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
  path = '{}{}_{}_{}_{}_{}_{}_{}_{}.png'.format(save_path,measurement_id,target1,value1,target2,value2,target3,value3,picture_id)
  plt.savefig(path)
  time.sleep(0.01)
  print(path)

def loop_files(files):
  
  for file in files:
    all_plots_user = []
    user_id = file.split('.')[0]
    df = pd.read_csv(str(savedir + str(user_id)) + ".csv",index_col=0)
    measurement_list = list(df['measurement_id'].unique())
    print('\n Running for user: ' + str(user_id))
    #Make user's plot folder
    save_path = plot_path + str(user_id) +'/'
    try: 
      os.mkdir(save_path) 
    except OSError as error: 
      print(error) 
    #files_user = [f for f in listdir(save_path) if isfile(join(save_path, f))]

    #Check for processed measurements in save dir
    procfiles = glob.glob((save_path + '*.png'))

    if set(all_plots_user) == set(procfiles) and len(procfiles)>=1: #If all user files are processed
      print('All files plotted for user: ' + str(user_id))
      continue

    measurement_list = list(set(df['measurement_id']))

    all_plot_names = []
    for i in range(len(measurement_list)):
      torch.cuda.empty_cache()
      measurement_id = measurement_list[i]
      ts = df[df['measurement_id']==measurement_id]
      # print(measurement_id)
      print(measurement_id)

      res = [k for k in procfiles if measurement_id in k] # return all processed files for current measurement
      all_plot_names = gen_labels(ts, save_path,measurement_id,target1,target2,target3, overlap_win)
      
      all_plots_user.append(all_plot_names)

      filepaths = []
      for filepath in pathlib.Path(save_path).glob('**/*'):
          filepaths.append(str(filepath.absolute()))
      if set(all_plot_names) == set(res): #If all measurement files are processed
        print('All files plotted for measurement: ' + str(measurement_id))
        continue

      for start_point in range(0,len(ts.measurement_id),overlap_win):     # each loop will generate a picture and save it in save_path 
        ts_slice, value1, value2, value3,picture_id = select_df(df,measurement_id,start_point,window_size,target1,target2,target3)        
        if any(str(picture_id) in s for s in res):
          print('Already plotted')
          time.sleep(0.1)
          clear()
          continue
        ts_p = ts_slice[col_names]
        # ts_p = ts_slice[col_names]
        print(picture_id)
        time.sleep(0.01)
        clear_output()
        plot_ts(ts_p, save_path,measurement_id,target1,value1,target2,value2,target3,value3,picture_id)
        ts_slice = None
        clear()
    gc.collect()

loop_files(files)

/usr/local/lib/python3.6/dist-packages/pandas/plotting/_matplotlib/tools.py:184: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(**fig_kw)


In [3]:
from IPython.display import clear_output
import glob
from os import listdir
from os.path import isfile, join
import pandas as pd
import random
import torch
import os
import tqdm
import gc
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pathlib
import time
plt.ioff()
import sys


#Process all users in a loop - checks for processed files if it crashes
savedir = "/content/drive/My Drive/BEAT_PD/Data/CIS-PD/training_data/HAR/merged/"
files = [f for f in listdir(savedir) if isfile(join(savedir, f))]

In [4]:
files

['1004.csv',
 '1006.csv',
 '1019.csv',
 '1020.csv',
 '1023.csv',
 '1032.csv',
 '1034.csv',
 '1038.csv',
 '1039.csv',
 '1043.csv',
 '1044.csv',
 '1046.csv',
 '1048.csv',
 '1049.csv',
 '1051.csv',
 '1007.csv']